In [ ]:
#!pip install catboost
#import catboost

In [ ]:
#!pip install shap
#import shap

In [ ]:
from io import IncrementalNewlineDecoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib Inline

import seaborn as sns
sns.set()

from catboost import CatBoostRegressor, Pool, cv
from catboost import MetricVisualizer

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from scipy.stats import boxcox
from os import listdir

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import shap
shap.initjs()

ModuleNotFoundError: No module named 'catboost'

In [ ]:
data = pd.read_csv("/content/data.csv", encoding="ISO-8859-1")
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/data.csv'

In [ ]:
data.head()

In [ ]:
missing_percentage = data.isnull().sum()/data.shape[0]*100
missing_percentage

In [ ]:
data[data["Description"].isnull()].head()

In [ ]:
data[data["Description"].isnull()].CustomerID.isnull().value_counts()

In [ ]:
data[data["Description"].isnull()].UnitPrice.isnull().value_counts()

In [ ]:
data[data.CustomerID.isnull()	].head()

In [ ]:
data.loc[data.CustomerID.isnull(),["UnitPrice","Quantity"]].describe()

In [ ]:
data.loc[data.Description.isnull()==False, "lowercase_descriptions"]=data.loc[
    data.Description.isnull()==False,"Description"
    ].apply(lambda l: l.lower())
data.lowercase_descriptions.dropna().apply(lambda l: np.where("nan" in l, True, False)).value_counts()


In [ ]:
data.lowercase_descriptions.dropna().apply(lambda l: np.where("" == l, True,False)).value_counts()

In [ ]:
data.loc[data.lowercase_descriptions.isnull()==False, "lowercase_descriptions"] = data.loc[
    data.lowercase_descriptions.isnull()==False, "lowercase_descriptions"
].apply(lambda l: np.where("nan" in l, None, l))

In [ ]:
data= data.loc[(data.CustomerID.isnull()==False) & (data.lowercase_descriptions.isnull()==False)].copy()

In [ ]:
data.isnull().sum().sum()

In [ ]:
data["InvoiceDate"]=pd.to_datetime(data.InvoiceDate,cache=True)

data.InvoiceDate.max()-data.InvoiceDate.min()

In [ ]:
print("datefile starts with time point{}".format(data.InvoiceDate.min()))
print("datefile ends with time point{}".format(data.InvoiceDate.max()))

In [ ]:
data.InvoiceNo.nunique()

In [ ]:
data["IsCancelled"] = np.where(data.InvoiceNo.apply(lambda l: l[0]=="C"),True,False)
data.IsCancelled.value_counts()/data.shape[0]*100

In [ ]:
data.loc[data.IsCancelled==True].describe()

In [ ]:
data=data.loc[data.IsCancelled==False].copy()
data=data.drop("IsCancelled",axis=1)


In [ ]:
data.StockCode.nunique()

In [ ]:


# Calculate the counts of each StockCode
stockcode_counts = data['StockCode'].value_counts().sort_values(ascending=False)

# Create subplots
fig, ax = plt.subplots(2, 1, figsize=(20, 15))

# Plot the top 20 most common StockCodes as a bar plot
sns.barplot(x=stockcode_counts.iloc[0:20].index,
            y=stockcode_counts.iloc[0:20].values,
            ax=ax[0],
            palette="Oranges_r")
ax[0].set_ylabel("Counts")
ax[0].set_xlabel("Stockcode")
ax[0].set_title("Which stockcodes are most common?")

# Plot the distribution of the percentage of data with each StockCode
sns.distplot(np.round(stockcode_counts / data.shape[0] * 100, 2),
             kde=False,
             bins=20,
             ax=ax[1],
             color="Orange")
ax[1].set_title("How seldom are stockcodes?")
ax[1].set_xlabel("% of data with this stockcode")
ax[1].set_ylabel("Frequency")

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
def counts_numeric_chats(l):
  return sum(1 for c in l if c.isdigit())

data['StockCodeLenth']=data.StockCode.apply(lambda l: len(l))
data['nNumericStockCode']=data.StockCode.apply(lambda l: counts_numeric_chats(l))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,15))

sns.countplot(data['StockCodeLenth'], palette='#ff6200',ax=ax[0])
sns.countplot(data['nNumericStockCode'],palette='#ff6200',ax=ax[1])

ax[0].set_xlabel("lenth of stockcode")
ax[1].set_xlabel("number of numeric chars in stockcodes")

plt.show()

In [ ]:
data.loc[data["nNumericStockCode"]<5].lowercase_descriptions.value_counts()